# Update push streaming Python example


Version: 15 Sept 2022

https://docs.imply.io/polaris/api-stream/

Updates the push streaming Coinbase example to account for the following:
 * v2 APIs
 * API keys

In [29]:
import datetime
import json
import os
import requests
import time

In [8]:
# Read API key from environment variable.
# API keys do not expire.
# Do not supply an API key in production scripts, and
# do not check them into version control systems.
# For more information, see https://docs.imply.io/polari/api-keys
API_KEY = os.getenv("POLARIS_API_KEY")

# Replace with the name of your organization
ORG_NAME = "imply" # TODO change to example

# Supply the push streaming connection name in the following string variable
CONN_NAME = "coinbase" # TODO change

In [34]:
# TODO do not include
print(API_KEY)

pok_VIpGJRKmdPpa3VqjJMVY3822hQNIitx4GawrOUDugWZMX1tfRnunFcKenZUZBJJVyO


In [19]:
# Polaris Events API
EVENTS_ENDPOINT = f"https://{ORG_NAME}.api.eng.imply.io/v1/events/{CONN_NAME}" # TODO remove eng

In [10]:
# TODO test
EVENTS_ENDPOINT

'https://imply.api.imply.io/v1/events/coinbase'

In [16]:
# TODO test
access_token = "eyJhbGciOiJSUzI1NiIsInR5cCIgOiAiSldUIiwia2lkIiA6ICJ6LVF2cnYyUDIwMk5JUnpsVHpENlc0eG41bkp5cHdMaGFmNnU1UUdEX3hVIn0.eyJleHAiOjE2NjMzNTE5NDcsImlhdCI6MTY2Mjc0NzE0NywianRpIjoiNTQ5M2UxMTgtY2RjYS00MjBjLThiMjEtMWRjN2ZiM2QzZjVjIiwiaXNzIjoiaHR0cHM6Ly9pZC5lbmcuaW1wbHkuaW8vYXV0aC9yZWFsbXMvaW1wbHkiLCJhdWQiOiJjbHVzdGVyLTNiNTM4YTcyLTg0NGYtNGEwOS04ODNkLTYzMmU5MTMwMWU5NS1pbXBseS1kcnVpZC1hdXRoZW50aWNhdG9yIiwic3ViIjoiYTI4ZjM1MGYtOWNlNi00ZDRiLThmM2ItMzk5MGFjOTc4MmQxIiwidHlwIjoiQmVhcmVyIiwiYXpwIjoiZG9jcy1kZW1vIiwiYWNyIjoiMSIsInJlYWxtX2FjY2VzcyI6eyJyb2xlcyI6WyJNYW5hZ2VJbmdlc3Rpb25Kb2JzIiwiQWNjZXNzU2NhbGluZyIsIk1hbmFnZUFwaUtleXMiLCJNYW5hZ2VEYXNoYm9hcmRzIiwiQWNjZXNzUXVlcmllcyIsIk1hbmFnZVN0cmVhbXMiLCJNYW5hZ2VUYWJsZXMiLCJNYW5hZ2VSZXBvcnRzIiwiQWRtaW5pc3RlclJlcG9ydHMiLCJNYW5hZ2VBbGVydHNXZWJob29rcyIsIkFjY2Vzc1Zpc3VhbGl6YXRpb24iLCJBY2Nlc3NSZXBvcnRzIiwiQWNjZXNzTWV0cmljcyIsIkFjY2Vzc0Rvd25sb2FkRGF0YSIsIk1hbmFnZVByb2plY3RzIiwiQWRtaW5pc3RlclF1ZXJpZXMiLCJBY2Nlc3NBbGVydHMiLCJNYW5hZ2VGaWxlcyIsIk1hbmFnZURhdGFDdWJlcyIsIkFkbWluaXN0ZXJTY2FsaW5nIiwiQWNjZXNzUXVlcnlSYXdEYXRhIiwiTWFuYWdlQWxlcnRzIiwiQWNjZXNzRG93bmxvYWRMYXJnZURhdGEiLCJBY2Nlc3NNb25pdG9yUXVlcmllcyIsIlZpZXdUYWJsZXMiXX0sInJlc291cmNlX2FjY2VzcyI6eyJjbHVzdGVyLTNiNTM4YTcyLTg0NGYtNGEwOS04ODNkLTYzMmU5MTMwMWU5NS1pbXBseS1kcnVpZC1hdXRoZW50aWNhdG9yIjp7InJvbGVzIjpbIlJlYWREYXRhU291cmNlcyJdfX0sInNjb3BlIjoiZW1haWwgcHJvZmlsZSIsImNsaWVudElkIjoiZG9jcy1kZW1vIiwiY2xpZW50SG9zdCI6IjEwLjEwMy4xMDEuNTUiLCJlbWFpbF92ZXJpZmllZCI6ZmFsc2UsInByZWZlcnJlZF91c2VybmFtZSI6InNlcnZpY2UtYWNjb3VudC1kb2NzLWRlbW8iLCJjbGllbnRBZGRyZXNzIjoiMTAuMTAzLjEwMS41NSIsImFjY3QiOiIyMmU2NTUzNC1kMDg4LTQ4NTgtOGIxZS01ZTRkY2M4NWQ5ODYifQ.LueCpqzN_AqZe8NZ2F6NRMrWTWwTv8FuGRPorEOU8Gd9OPNIsAn3c1okgqWxfENjysHI4f8K_uYjqOsZUPeEAFmY9fuZCQc9sgDjzcRVPdIl5F1rDR2FQGxVMXs0xOZ4e5-XJeMl0hMpNFCMYP7KENajRF-eJnsyBUQveClg6lIk7qKH2WpGVx_O38T6JcUwwiMqddUbAgLUIAfOvfp6DrVZVDTNBOG6v43bbXpz_TP0XRSQSvL3J3r7El5vihEjJonhL63ZiHk1yCarLcVZf7IFE8V6xy-D8aEGXVg8a5Vh04CiMxXj14KP3eF4h6mEyshipXQsUbOqyP1b46_SeA"

In [31]:
# Called by push_data to send an event to the Polaris connection
def make_post(url, json):
    def do_post():
        headers = {
            'Authorization': f'Basic {API_KEY}:',
            #"Authorization": "Bearer {token}".format(token=access_token),
            "Content-Type": "application/json"
        }
        return requests.post(url, headers=headers, data=json)

    response = do_post()

    # Raise an exception on response errors at this point
    response.raise_for_status()

    return response

# Send a POST request and print HTTP response
def push_data(data):
    response = make_post(EVENTS_ENDPOINT, data)
    print(response.status_code, response.reason)
    print(response.text)
    print("----")


while True:
    
    # Get the price of bitcoin and push it
    btc_resp = requests.get("https://api.coinbase.com/v2/prices/spot?currency=USD")
    btc_resp.raise_for_status()

    # Try the push. If you get a 401 error, verify that your API key has the right permissions.
    push_data(json.dumps({
        "event_time": datetime.datetime.utcnow().isoformat(),
        "price": float(btc_resp.json()["data"]["amount"]),
    }))
    time.sleep(1)

200 OK

----
200 OK

----
200 OK

----


KeyboardInterrupt: 

In [5]:
# TODO test =========================================
btc_resp = requests.get("https://api.coinbase.com/v2/prices/spot?currency=USD")

In [6]:
print(btc_resp.json()["data"]["amount"])

19850.66


In [7]:
print(btc_resp.json()["data"])

{'base': 'BTC', 'currency': 'USD', 'amount': '19850.66'}


In [24]:
datetime.datetime.utcnow().isoformat()

'2022-09-15T22:57:00.354689'

In [26]:
{
    "event_time": datetime.datetime.utcnow().isoformat(),
    "price": float(btc_resp.json()["data"]["amount"]),
}

{'event_time': '2022-09-15T22:59:32.894144', 'price': 19631.46}